# Moscow Marathon Full Results 2018


## 1_full_results_mm_2018.csv

Columns : {Bib, finish_time_sec, finish_time_result, race, pace_sec, pace(minpkm), pace(kmph), half_pace_sec, half_pace(minpkm), half_pace(kmph), gender_en, agev name_en, location_city_ru, location_city_en, country_code_alpha_3, flag_DNF, flag_all_split_exist, race_uniform_index}

Data : {1, 8911, 2h 28min 31sec, 42.195 km, 211.1861595, 3:31 min/km 17.0 km/h, 208.3185212, 3:28 min/km, 17.3 km/h, Female, 30, Sardana Trofimova, –Ø–∫—É—Ç—Å–∫, Yakutsk, RUS, 0, 1, 0.000132899}

## 1_split_results_mm_2018.csv

Columns : {bib, split_name, split, split_time_sec, split_time_result, split_pace_sec, split_pace(minpkm), split_pace(kmph), split_uniform_index}

Data : {11, Kirui, Geoffrey, 24, M, Keringet, KEN, 0:15:25, 0:30:28, 0:45:44, 1:01:15, 1:04:35, 1:16:59, 1:33:01, 1:48:19, 2:02:53, 0:04:57, - 2:09:37, 1, 1, 11}

In [143]:
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd

import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  

In [145]:
full_df  = pd.read_csv('./data/1_full_results_mm_2018.csv')
split_df = pd.read_csv('./data/1_split_results_mm_2018.csv')

In [147]:
full_df.head()

,bib,finish_time_sec,finish_time_result,race,pace_sec,pace(minpkm),pace(kmph),half_pace_sec,half_pace(minpkm),half_pace(kmph),gender_en,age,name_en,location_city_ru,location_city_en,country_code_alpha_3,flag_DNF,flag_all_split_exist,race_uniform_index
0,1,8911.0,2h 28min 31sec,42.195 km,211.186159,3:31 min/km,17.0 km/h,208.318521,3:28 min/km,17.3 km/h,Female,30,Sardana Trofimova,Якутск,Yakutsk,RUS,0,1,0.000133
1,2,9308.0,2h 35min 08sec,42.195 km,220.594857,3:40 min/km,16.3 km/h,212.157839,3:32 min/km,17.0 km/h,Female,35,Tat'yana Arkhipova,Чебоксары,Cheboksary,RUS,0,1,0.002556
2,3,8122.0,2h 15min 22sec,42.195 km,192.487262,3:12 min/km,18.7 km/h,189.880318,3:09 min/km,19.0 km/h,Male,31,Stepan Kiselev,Казань,Kazan',RUS,0,1,0.000397
3,4,8467.0,2h 21min 07sec,42.195 km,200.663586,3:20 min/km,17.9 km/h,186.894182,3:06 min/km,19.3 km/h,Male,36,Dmitriy Safronov,Москва,Moskva,RUS,0,1,0.009003
4,5,8738.0,2h 25min 38sec,42.195 km,207.086148,3:27 min/km,17.4 km/h,194.904610,3:14 min/km,18.5 km/h,Male,42,Grigoriy Andreev,Уфа,Ufa,RUS,0,1,0.006439


In [149]:
split_df.head()

,bib,split_name,split,split_time_sec,split_time_result,split_pace_sec,split_pace(minpkm),split_pace(kmph),split_uniform_index
0,1,5 km,5.0000,1057,17min 37sec,211.400000,3:31 min/km,17.0 km/h,1.214334e-07
1,1,10 km,10.0000,2112,35min 12sec,211.000000,3:30 min/km,17.1 km/h,1.017882e-09
2,1,15 km,15.0000,3149,52min 29sec,207.400000,3:27 min/km,17.4 km/h,1.254794e-05
3,1,Half marathon,21.0975,4395,1h 13min 15sec,204.346043,3:24 min/km,17.6 km/h,9.282250e-05
4,1,25 km,25.0000,5258,1h 27min 38sec,221.140295,3:41 min/km,16.3 km/h,9.987022e-06


In [151]:
# 구간별 split 시간 pivot
split_time = split_df.pivot_table(index='bib', columns='split_name', values='split_time_sec')
split_time.columns = [col.replace(' ', '').replace('km', 'K') if 'km' in col else col.lower() for col in split_time.columns]
split_time.reset_index(inplace=True)

In [153]:
# pace(p) 데이터를 "분:초" 문자열로 wide-format 생성
def sec_to_minsec(sec):
    if pd.isnull(sec):
        return None
    minutes = int(sec // 60)
    seconds = int(sec % 60)
    return f"{minutes}:{seconds:02d}"

In [155]:
# split_pace_sec → wide-format + 문자열 변환
split_pace = split_df.pivot_table(index='bib', columns='split_name', values='split_pace_sec')
split_pace.columns = [col.replace(' km', '').replace('Half marathon', '21').replace('Marathon', '42') + 'p' for col in split_pace.columns]
split_pace = split_pace.applymap(sec_to_minsec)
split_pace.reset_index(inplace=True)

/var/folders/6m/mc6f2cbd0y96kqwywtrhvss40000gn/T/ipykernel_75190/2134712935.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  split_pace = split_pace.applymap(sec_to_minsec)


In [157]:
# full_df에서 필요한 컬럼 정리
rename_map = {
    'bib': 'Bib',
    'name_en': 'Name',
    'age': 'Age',
    'gender_en': 'M/F',
    'location_city_en': 'City',
    'country_code_alpha_3': 'Country',
    'finish_time_sec': 'Official Time Sec'
}
reduced_df = full_df[list(rename_map.keys())].rename(columns=rename_map)

# 시간(hh:mm:ss) 추가
reduced_df['Official Time Hour'] = pd.to_timedelta(reduced_df['Official Time Sec'], unit='s')


In [159]:
# 데이터 병합
merged = reduced_df.merge(split_time, how='left', left_on='Bib', right_on='bib')
merged = merged.merge(split_pace, how='left', on='bib')
merged.drop(columns=['bib'], inplace=True)


In [161]:
# 정렬 및 출력 확인
columns_order = ['Bib', 'Name', 'Age', 'M/F', 'City', 'Country'] + \
                ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Marathon'] + \
                ['Official Time Hour'] + \
                ['5p', '10p', '15p', '20p', '21p', '25p', '30p', '35p', '40p', '42p']

# 컬럼 누락 방지 (존재하는 컬럼만 포함)
columns_order = [col for col in columns_order if col in merged.columns]

# 결과 저장 or 출력
final_df = merged[columns_order]
print(final_df.head())

   Bib                Name  Age     M/F        City Country      5K     10K  \
0    1   Sardana Trofimova   30  Female     Yakutsk     RUS  1057.0  2112.0   
1    2  Tat'yana Arkhipova   35  Female  Cheboksary     RUS  1059.0  2115.0   
2    3      Stepan Kiselev   31    Male      Kazan'     RUS   947.0  1899.0   
3    4    Dmitriy Safronov   36    Male      Moskva     RUS   945.0  1876.0   
4    5    Grigoriy Andreev   42    Male         Ufa     RUS   976.0  1958.0   

      15K     25K  ...  Official Time Hour    5p   10p   15p   21p   25p  \
0  3149.0  5258.0  ...     0 days 02:28:31  3:31  3:31  3:27  3:24  3:41   
1  3188.0  5386.0  ...     0 days 02:35:08  3:31  3:31  3:34  3:31  3:53   
2  2867.0  4796.0  ...     0 days 02:15:22  3:09  3:10  3:13  3:06  3:22   
3  2814.0  4752.0  ...     0 days 02:21:07  3:09  3:06  3:07  3:05  3:27   
4  2933.0  4949.0  ...     0 days 02:25:38  3:15  3:16  3:15  3:13  3:34   

    30p   35p   40p   42p  
0  3:32  3:28  3:36  3:33  
1  3:44  3:4

In [163]:
# M/F로 표준화
final_df['M/F'] = final_df['M/F'].map({'Male': 'M', 'Female': 'F'})

/var/folders/6m/mc6f2cbd0y96kqwywtrhvss40000gn/T/ipykernel_75190/3973932359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['M/F'] = final_df['M/F'].map({'Male': 'M', 'Female': 'F'})


In [165]:
total_participants = len(final_df)
print(f"총 마라톤 참가자 수: {total_participants}명")

총 마라톤 참가자 수: 21662명


In [167]:
print(final_df.head())

   Bib                Name  Age M/F        City Country      5K     10K  \
0    1   Sardana Trofimova   30   F     Yakutsk     RUS  1057.0  2112.0   
1    2  Tat'yana Arkhipova   35   F  Cheboksary     RUS  1059.0  2115.0   
2    3      Stepan Kiselev   31   M      Kazan'     RUS   947.0  1899.0   
3    4    Dmitriy Safronov   36   M      Moskva     RUS   945.0  1876.0   
4    5    Grigoriy Andreev   42   M         Ufa     RUS   976.0  1958.0   

      15K     25K  ...  Official Time Hour    5p   10p   15p   21p   25p  \
0  3149.0  5258.0  ...     0 days 02:28:31  3:31  3:31  3:27  3:24  3:41   
1  3188.0  5386.0  ...     0 days 02:35:08  3:31  3:31  3:34  3:31  3:53   
2  2867.0  4796.0  ...     0 days 02:15:22  3:09  3:10  3:13  3:06  3:22   
3  2814.0  4752.0  ...     0 days 02:21:07  3:09  3:06  3:07  3:05  3:27   
4  2933.0  4949.0  ...     0 days 02:25:38  3:15  3:16  3:15  3:13  3:34   

    30p   35p   40p   42p  
0  3:32  3:28  3:36  3:33  
1  3:44  3:40  3:55  3:58  
2  3:18 

In [171]:
# 결측치 개수 확인
missing = final_df.isnull().sum()
missing = missing[missing > 0]
print("결측치가 있는 컬럼:")
print(missing.sort_values(ascending=False))

🔍 결측치가 있는 컬럼:
42p                   12960
35K                   12957
35p                   12957
40K                   12946
40p                   12946
30p                   12932
30K                   12932
21p                   12865
25K                   12863
25p                   12863
15K                   12756
15p                   12756
Official Time Hour      277
5K                      121
5p                      121
10p                     120
10K                     120
dtype: int64


In [173]:
# 결측치가 하나라도 있는 행 제거
clean_df = final_df.dropna()

# 1부터 시작하는 새로운 인덱스 컬럼 추가
clean_df.insert(0, 'No', range(1, len(clean_df) + 1))

# 확인
print(clean_df.head())

   No  Bib                Name  Age M/F        City Country      5K     10K  \
0   1    1   Sardana Trofimova   30   F     Yakutsk     RUS  1057.0  2112.0   
1   2    2  Tat'yana Arkhipova   35   F  Cheboksary     RUS  1059.0  2115.0   
2   3    3      Stepan Kiselev   31   M      Kazan'     RUS   947.0  1899.0   
3   4    4    Dmitriy Safronov   36   M      Moskva     RUS   945.0  1876.0   
4   5    5    Grigoriy Andreev   42   M         Ufa     RUS   976.0  1958.0   

      15K  ...  Official Time Hour    5p   10p   15p   21p   25p   30p   35p  \
0  3149.0  ...     0 days 02:28:31  3:31  3:31  3:27  3:24  3:41  3:32  3:28   
1  3188.0  ...     0 days 02:35:08  3:31  3:31  3:34  3:31  3:53  3:44  3:40   
2  2867.0  ...     0 days 02:15:22  3:09  3:10  3:13  3:06  3:22  3:18  2:49   
3  2814.0  ...     0 days 02:21:07  3:09  3:06  3:07  3:05  3:27  3:25  3:29   
4  2933.0  ...     0 days 02:25:38  3:15  3:16  3:15  3:13  3:34  3:33  3:33   

    40p   42p  
0  3:36  3:33  
1  3:55  3:5

In [177]:
# 결측치 개수 확인
missing = clean_df.isnull().sum()
missing = missing[missing > 0]
print("결측치가 있는 컬럼:")
print(missing.sort_values(ascending=False))

🔍 결측치가 있는 컬럼:
Series([], dtype: int64)
